In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from autokeras import StructuredDataRegressor
import sys
sys.path.append('../scripts')
from feature_transform import dates_encoder, merge_path
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential

In [69]:
train = pd.read_csv('../data/flights_train.csv', parse_dates=['flight_date'])
X_test = pd.read_csv('../data/flights_Xtest.csv', parse_dates=['flight_date'])

In [70]:
train

,flight_date,from,to,avg_weeks,target,std_weeks
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159
...,...,...,...,...,...,...
8891,2011-11-24,DEN,PHX,11.045455,9.795363,7.791798
8892,2012-09-01,ATL,DEN,11.000000,9.429776,7.937254
8893,2012-08-10,ORD,EWR,9.476190,12.029971,6.903760
8894,2012-07-03,ORD,IAH,10.250000,9.381645,7.107261


In [71]:
train = merge_path(train)
X_test = merge_path(X_test)

In [72]:
date_encoder = FunctionTransformer(dates_encoder)
date_cols = ["flight_date"]

categorical_encoder = OneHotEncoder(handle_unknown="ignore", sparse = False)
categorical_cols = ["path"]

In [73]:
preprocessor = make_column_transformer(
        (date_encoder, date_cols),
        (categorical_encoder, categorical_cols),
        remainder='passthrough')

In [74]:
train_X, val_X, train_y, val_y = train_test_split(train.drop(['target'], 1), train[['target']], test_size=0.2, random_state=42)

train_X_processed = preprocessor.fit_transform(train_X)
train_y_processed = train_y['target'].values
val_X_processed = preprocessor.transform(val_X)
val_y_processed = val_y['target'].values

In [75]:
from tensorflow.compat.v1.keras import backend as K
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
search = StructuredDataRegressor(max_trials=15, loss=root_mean_squared_error)
search.fit(train_X_processed, train_y_processed)

INFO:tensorflow:Reloading Oracle from existing project .\structured_data_regressor\oracle.json


In [ ]:
search.evaluate(val_X_processed, val_y_processed, verbose=0)

In [ ]:
yhat = search.predict(X_new)

In [77]:
# use autokeras to find a model for the insurance dataset
from numpy import asarray
from pandas import read_csv
from sklearn.model_selection import train_test_split
from autokeras import StructuredDataRegressor
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/auto-insurance.csv'
dataframe = read_csv(url, header=None)
print(dataframe.shape)
# split into input and output elements
data = dataframe.values
data = data.astype('float32')
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# define the search
search = StructuredDataRegressor(max_trials=15, loss='mean_absolute_error')
# perform the search
search.fit(x=X_train, y=y_train, verbose=0)
# evaluate the model

(63, 2)
(63, 1) (63,)
(42, 1) (21, 1) (42,) (21,)
INFO:tensorflow:Reloading Oracle from existing project .\structured_data_regressor\oracle.json


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 223: invalid start byte

In [ ]:
mae, _ = search.evaluate(X_test, y_test, verbose=0)
print('MAE: %.3f' % mae)
# use the model to make a prediction
X_new = asarray([[108]]).astype('float32')
yhat = search.predict(X_new)
print('Predicted: %.3f' % yhat[0])
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()
# save the best performing model to file
model.save('model_insurance.h5')